In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score

In [2]:
! ls drive/MyDrive/ShoppingPulse/datasets/

content_based_train_item_metadata.pkl  processed
interactions_test_data1.parquet        raw
interactions_test_data.parquet	       svd_trainset.pkl
interactions_training_data1.parquet    test_metadata.parquet
interactions_training_data.parquet     train_metadata.parquet
interactions_validation_data1.parquet  train_reviews.parquet
interactions_validation_data.parquet   valid_metadata.parquet


In [3]:
train_item_metadata = pd.read_parquet("drive/MyDrive/ShoppingPulse/datasets/train_metadata.parquet")


In [34]:
test_item_metadata = pd.read_parquet("drive/MyDrive/ShoppingPulse/datasets/test_metadata.parquet")
valid_item_metadata = pd.read_parquet("drive/MyDrive/ShoppingPulse/datasets/valid_metadata.parquet")

In [4]:
train_interaction_data = pd.read_parquet("drive/MyDrive/ShoppingPulse/datasets/interactions_training_data1.parquet")


In [43]:
valid_interaction_data = pd.read_parquet("drive/MyDrive/ShoppingPulse/datasets/interactions_validation_data1.parquet")


In [62]:
test_interaction_data = pd.read_parquet("drive/MyDrive/ShoppingPulse/datasets/interactions_test_data1.parquet")


In [16]:
train_interaction_data.head()

,user_id,parent_asin,rating,timestamp,date_time,category
0,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B09BF693L6,5.0,1547316703224,2019-01-12 18:11:43.224,Automotive
1,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B01KP4X3UO,5.0,1575402993972,2019-12-03 19:56:33.972,Automotive
2,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B07Q3HTGC7,4.0,1589395842176,2020-05-13 18:50:42.176,Automotive
3,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B08X1F42NG,4.0,1590212444932,2020-05-23 05:40:44.932,Automotive
4,AFZUK3MTBIBEDQOPAK3OATUOUKLA,B0BZJ6PFBB,3.0,1590215175314,2020-05-23 06:26:15.314,Automotive


In [5]:
train_item_metadata['description_processed'] = train_item_metadata['description'].apply(lambda x: ' '.join(x))
train_item_metadata['features_processed'] = train_item_metadata['features'].apply(lambda x: ' '.join(x))

In [6]:
train_item_metadata['description_processed'] = train_item_metadata['description_processed'].str.lower().str.replace('[^\w\s]', '')
train_item_metadata['features_processed'] = train_item_metadata['features_processed'].str.lower().str.replace('[^\w\s]', '')

In [7]:
train_item_metadata['title_processed'] = train_item_metadata['title'].str.lower().str.replace('[^\w\s]', '')

In [9]:
train_item_metadata.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,description_processed,features_processed,title_processed
14,Automotive,"CURT 25080 Trailer Coupler Pin, 1/4-Inch Diame...",4.7,5640,[VERSATILE. The 1/4-inch diameter and 2-3/4-in...,[CURT safety pins go beyond the simple functio...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",CURT,"[Automotive, Exterior Accessories, Trailer Acc...","{""Manufacturer"": ""Curt Manufacturing"", ""Brand""...",B001GN3H5A,None,None,None,curt safety pins go beyond the simple function...,versatile. the 1/4-inch diameter and 2-3/4-inc...,"curt 25080 trailer coupler pin, 1/4-inch diame..."
29,Automotive,KA Depot for Cobalt 2005-2010 15289946 GM12511...,4.2,4,"[Brand New OE Replacement Part By KA Depot, Fi...",[Fits Cobalt 2005-2010],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",KA Depot,"[Automotive, Replacement Parts, Body & Trim, B...","{""Brand"": ""KA Depot"", ""Exterior Finish"": ""Plas...",B07LGZBPF4,None,None,None,fits cobalt 2005-2010,brand new oe replacement part by ka depot fits...,ka depot for cobalt 2005-2010 15289946 gm12511...
40,Automotive,Farmogo Tesla Door Lights Logo Projector Puddl...,4.6,1042,[【Compatibility】-The Tesla door light logo can...,[Tesla Logo Projector puddle lights],32.99,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Farmogo,"[Automotive, Lights & Lighting Accessories, Ac...","{""Item Dimensions LxWxH"": ""1.77 x 1.5 x 0.98 i...",B09GKDZJWT,None,None,None,tesla logo projector puddle lights,【compatibility】-the tesla door light logo can ...,farmogo tesla door lights logo projector puddl...
42,Automotive,Permatex 81630 Ultra Red High Temperature Gask...,4.7,6816,"[High temperature performance, adhesion and fl...",[Permatex’s Ultra Red Gasket Maker is ideal fo...,7.27,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Permatex,"[Automotive, Car Care, Fillers, Adhesives & Se...","{""Manufacturer"": ""Permatex"", ""Brand"": ""Permate...",B0BW4HHFX2,None,None,None,permatex’s ultra red gasket maker is ideal for...,"high temperature performance, adhesion and fle...",permatex 81630 ultra red high temperature gask...
49,Automotive,Salusy for Explorer 2022 2021 2020 Interior Ac...,4.6,124,[【Applicable models】: The central armrest stor...,"[Compatible:, Ford Explorer 2020 2021 2022, Fu...",13.66,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Salusy,"[Automotive, Replacement Parts, Body & Trim, T...","{""Manufacturer"": ""Salusy"", ""Brand"": ""Salusy"", ...",B08DKKSCKY,None,None,None,compatible: ford explorer 2020 2021 2022 funct...,【applicable models】: the central armrest stora...,salusy for explorer 2022 2021 2020 interior ac...


In [8]:
# Preprocess text features using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000)  # Limit features for efficiency
title_tfidf = vectorizer.fit_transform(train_item_metadata['title_processed'])
description_tfidf = vectorizer.fit_transform(train_item_metadata['description_processed'])

In [9]:
# Combine features into a single feature matrix
combined_features = np.hstack((title_tfidf.toarray(), description_tfidf.toarray()))
#train_item_metadata['combined_features'] = list(combined_features)

In [10]:
train_item_metadata['combined_features'] = list(combined_features)

In [42]:
train_item_metadata.reset_index(drop = True, inplace = True)

In [11]:
import pickle

In [41]:
# Save trainset to drive
with open('drive/MyDrive/ShoppingPulse/datasets/content_based_train_item_metadata.pkl', 'wb') as file:
    pickle.dump(train_item_metadata, file)

print("train_item_metadata saved to train_item_metadata.pkl")

train_item_metadata saved to train_item_metadata.pkl


In [13]:
from sklearn.preprocessing import normalize

# Normalize combined features for efficient cosine similarity computation
item_features_normalized = normalize(combined_features)

In [32]:
item_features_normalized.shape

(437422, 4000)

In [15]:
def recommend_items(user_id, interaction_data, _item_metadata, top_n=10):
    user_interactions = interaction_data[interaction_data['user_id'] == user_id]
    user_interacted_items = user_interactions['parent_asin'].tolist()

    user_profile = _item_metadata[_item_metadata['parent_asin'].isin(user_interacted_items)]
    if user_profile.empty:
        return pd.DataFrame()  # Return empty DataFrame if user has no interactions

    user_profile_indices = user_profile.index
    user_profile_features = item_features_normalized[user_profile_indices].mean(axis=0).reshape(1, -1)

    similarities = cosine_similarity(user_profile_features, item_features_normalized).flatten()
    similar_item_indices = similarities.argsort()[-top_n:][::-1]

    recommended_items = _item_metadata.iloc[similar_item_indices]
    return recommended_items


In [40]:
# Example usage
user_id = 'AFZUK3MTBIBEDQOPAK3OATUOUKLA'
interaction_data = train_interaction_data
_item_metadata = train_item_metadata

recommended_items = recommend_items(user_id, interaction_data, _item_metadata)
print(recommended_items[['title', 'main_category', 'price']])

                                                    title  \
216994  365 by Whole Foods Market, Waffles Homestyle O...   
232004       Auntys Sticky Toffee - UK - 6.7 oz Pack of 2   
196087            As I Am CocoShea Whip, 8 oz (Pack of 2)   
203912         Three Pack Pirucream Can 155 grs./ 5.46 Oz   
295643             TIGI Bedhead Manipulator, 2 oz(2 pack)   
155329  Mentholatum No Mess Vaporizing Rub - 1.76 oz, ...   
212775            Emerald Jalapeno Cashews 5 oz ( 2 Pack)   
147181                      Stomach Rescue 4 OZ Pack of 6   
220085        McCormick Celery Flakes, 0.5 oz (Pack of 6)   
214013                   Junior Mints, 3.5 Oz (Pack of 4)   

                 main_category  price  
216994                 Grocery   3.19  
232004                 Grocery   None  
196087                 Grocery  25.18  
203912                 Grocery  19.85  
295643              All Beauty  29.34  
155329  Health & Personal Care  22.36  
212775                 Grocery  13.99  
147181  

In [49]:
recommended_items.head(2)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author,description_processed,features_processed,title_processed,combined_features
216994,Grocery,"365 by Whole Foods Market, Waffles Homestyle O...",4.4,3637,[365 by Whole Foods Market products give you t...,"[Our standards are what set us apart, and our ...",3.19,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['Toaster Oven & my quick gluten fre...,365 by Whole Foods Market,"[Grocery & Gourmet Food, Frozen, Breakfast Foo...","{""Product Dimensions"": ""9.8 x 2.55 x 4.85 inch...",B074H5LSZ3,None,None,None,"our standards are what set us apart, and our q...",365 by whole foods market products give you th...,"365 by whole foods market, waffles homestyle o...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
232004,Grocery,Auntys Sticky Toffee - UK - 6.7 oz Pack of 2,3.3,8,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",Aunty's,[],"{""Package Dimensions"": ""6.6 x 5.5 x 3.1 inches...",B089DQVX48,None,None,None,,,auntys sticky toffee - uk - 6.7 oz pack of 2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [44]:
valid_interaction_data.head()

,user_id,parent_asin,rating,timestamp,date_time,user_in_train,parent_asin_in_train,category
0,AFK2K22B75H7M7SSBHY7JTFSRIOA,B0C5HT6P91,5.0,1647399904070,2022-03-16 03:05:04.070,True,True,Grocery_and_Gourmet_Food
1,AGNDBV4WPUIBYZMZQ4W36Y4K5T3Q,B087WR6JBQ,5.0,1649987177569,2022-04-15 01:46:17.569,True,False,Health_and_Household
2,AGX6ETVMVVJ24NNOSIOQCRYN5WGQ,B095J4YL2H,1.0,1633728491868,2021-10-08 21:28:11.868,True,True,Health_and_Household
3,AG4PI66B7EP24A3W4PX22BCMGWIQ,B0928NF7JN,4.0,1629262962037,2021-08-18 05:02:42.037,True,True,Beauty_and_Personal_Care
4,AGVV7VEPDHTWS5XD2LZ7L7VV6ZSA,B0BNBNP53F,5.0,1649626779149,2022-04-10 21:39:39.149,True,True,Automotive


In [56]:
valid_users = valid_interaction_data[valid_interaction_data.user_in_train == True]['user_id'].sample(500).unique()

In [75]:
valid_interaction_data2 = valid_interaction_data[valid_interaction_data.user_id.isin(valid_users)]

In [58]:
%%time
recommended_items_dict = {}
for user in valid_users:
    recommended_items = recommend_items(user, interaction_data, _item_metadata, top_n=2000).parent_asin
    recommended_items_dict.update({user: recommended_items})

CPU times: user 1h 6min 23s, sys: 16min 27s, total: 1h 22min 50s
Wall time: 1h 2min 10s


In [59]:
# Save to drive
with open('drive/MyDrive/ShoppingPulse/datasets/content_based_recommended_items_dict.pkl', 'wb') as file:
    pickle.dump(recommended_items_dict, file)

print("recommended items saved to recommended_items_dict.pkl")

recommended items saved to recommended_items_dict.pkl


In [66]:
list(recommended_items_dict.keys())[:4]

['AH467MZYVCA2KHYUHW55LL4UXHTQ',
 'AEHKF57TVUTZRZPHNWTQKYJCV67Q',
 'AEP2FXAALFU6MYXL4OKNJOL5SXZQ',
 'AG244QLI63TXV574JXXGXQEJKMCA']

In [70]:
len(recommended_items_dict['AH467MZYVCA2KHYUHW55LL4UXHTQ'])

2000

In [71]:
import numpy as np
from collections import defaultdict

In [72]:
# Function to calculate Precision@K and Recall@K
def precision_recall_at_k(recommendations, ground_truth, k_values):
    precision_at_k = defaultdict(list)
    recall_at_k = defaultdict(list)

    for user_id, recommended_items in recommendations.items():
        if user_id in ground_truth:
            actual_items = set(ground_truth[user_id])
            for k in k_values:
                recommended_at_k = set(recommended_items[:k])
                num_relevant_and_recommended = len(actual_items & recommended_at_k)
                precision = num_relevant_and_recommended / k
                recall = num_relevant_and_recommended / len(actual_items)

                precision_at_k[k].append(precision)
                recall_at_k[k].append(recall)

    avg_precision_at_k = {k: np.mean(precision_at_k[k]) for k in k_values}
    avg_recall_at_k = {k: np.mean(recall_at_k[k]) for k in k_values}

    return avg_precision_at_k, avg_recall_at_k

In [73]:
# Extract ground truth data from test and validation datasets
def extract_ground_truth(data):
    ground_truth = defaultdict(list)
    for _, row in data.iterrows():
        ground_truth[row['user_id']].append(row['parent_asin'])
    return ground_truth

In [76]:
val_ground_truth = extract_ground_truth(valid_interaction_data)

In [77]:
k_values = [200, 1000, 2000]
val_precision_at_k, val_recall_at_k = precision_recall_at_k(recommended_items_dict, val_ground_truth, k_values)
for k in k_values:
    print(f'Precision@{k}: {val_precision_at_k[k]}, Recall@{k}: {val_recall_at_k[k]}')

Precision@200: 2e-05, Recall@200: 0.004
Precision@1000: 3.4e-05, Recall@1000: 0.034
Precision@2000: 2.4e-05, Recall@2000: 0.048


In [63]:
test_interaction_data.head()

,user_id,parent_asin,rating,timestamp,date_time,user_in_train,parent_asin_in_train,category
0,AF733RJYQMOLEG6H7TMWRIURKKRQ,B0B2HTTNBS,4.0,1674101553682,2023-01-19 04:12:33.682,True,True,Health_and_Household
1,AFJUCDSJQSL6MM2JRKVGV3XCG6GQ,B00AP70A4O,5.0,1658173518379,2022-07-18 19:45:18.379,True,True,Health_and_Household
2,AFYESKC7PEEQN46FVRC5KU5WYV6Q,B01685FCSU,5.0,1666756519406,2022-10-26 03:55:19.406,True,True,Kindle_Store
3,AGPAAWUN75RTFNA5E3NDVRV25WQA,B07X8GCW7R,5.0,1681053942526,2023-04-09 15:25:42.526,True,False,Automotive
4,AHIVX63P64Y45DDZ2MAEUMZLHSSA,B076RW5LKG,1.0,1663694684756,2022-09-20 17:24:44.756,True,True,Beauty_and_Personal_Care


In [64]:
test_users = test_interaction_data[test_interaction_data.user_in_train == True]['user_id'].sample(500).unique()

In [68]:
%%time
recommended_items_test_dict = {}
for user in test_users:
    recommended_items = recommend_items(user, interaction_data, _item_metadata, top_n=2000).parent_asin
    recommended_items_test_dict.update({user: recommended_items})

CPU times: user 1h 4min 45s, sys: 16min 10s, total: 1h 20min 55s
Wall time: 1h 18s


In [69]:
# Save to drive
with open('drive/MyDrive/ShoppingPulse/datasets/content_based_recommended_items_dict_test.pkl', 'wb') as file:
    pickle.dump(recommended_items_test_dict, file)

print("recommended items saved to content_based_recommended_items_dict_test.pkl")

recommended items saved to content_based_recommended_items_dict_test.pkl
